# Select a couple of simpler trips to compare

* Compare methodologies, which differ when handling more complex shapes
* But is it different even in simpler shapes?
* Start with Big Blue Bus and LA Metro

In [168]:
import geopandas as gpd
import pandas as pd
from siuba import *

from shared_utils import rt_dates, rt_utils
from segment_speed_utils import helpers
from segment_speed_utils.project_vars import SEGMENT_GCS, PROJECT_CRS

from prep_comparison import map_one_trip

analysis_date = rt_dates.DATES["sep2023"]

import folium

In [115]:
from calitp_data_analysis.calitp_color_palette import CALITP_CATEGORY_BOLD_COLORS

In [116]:
df_eric = gpd.read_parquet(f"{SEGMENT_GCS}speeds_eric_{analysis_date}.parquet")
df_tiff = gpd.read_parquet(f"{SEGMENT_GCS}speeds_tiff_{analysis_date}.parquet")
# speed_df = pd.read_parquet(
#     f"{SEGMENT_GCS}speeds_comparison_{analysis_date}.parquet")

In [226]:
def remove_interpolated_segments(df_eric):
    
    df_eric = df_eric >> group_by(_.trip_id) >> arrange(_.stop_sequence) >> mutate(last_seq = _.stop_sequence.shift(1)) >> ungroup()
    df_eric.last_seq = df_eric.last_seq.bfill()
    df_eric = (df_eric >> filter(_.stop_sequence.astype(int) == _.stop_sequence)
         >> filter(_.last_seq.astype(int) == _.last_seq)
         >> select(-_.last_seq)
    )
    return df_eric

In [227]:
remove_interpolated_segments(df_eric)

,shape_meters,stop_id,stop_name,geometry,shape_id,trip_id,stop_sequence,route_id,route_short_name,direction_id,...,meters_from_last,speed_from_last,delay_chg_sec,speed_mph,n_trips_shp,p50_mph,p20_mph,p80_mph,fast_slow_ratio,trips_per_hour
140512,2000.000000,None,None,"LINESTRING (150048.445 -452697.224, 149992.409...",8570002_JUNE23,10857000020744-JUNE23,1.0,857-13168,857,0.0,...,1999.954469,1.412397,NaN,3.159533,10,15.139748,12.001957,17.624052,1.468432,5.333333
140513,3000.000000,None,None,"LINESTRING (149899.216 -450902.947, 149898.711...",8570002_JUNE23,10857000020744-JUNE23,1.0,857-13168,857,0.0,...,1000.000000,6.493506,NaN,14.525974,10,15.139748,12.001957,17.624052,1.468432,5.333333
140514,2000.000000,None,None,"LINESTRING (150048.445 -452697.224, 149992.409...",8570002_JUNE23,10857000020825-JUNE23,1.0,857-13168,857,0.0,...,1999.954469,5.390713,NaN,12.059025,10,15.139748,12.001957,17.624052,1.468432,5.333333
140515,3000.000000,None,None,"LINESTRING (149899.216 -450902.947, 149898.711...",8570002_JUNE23,10857000020825-JUNE23,1.0,857-13168,857,0.0,...,1000.000000,12.500000,NaN,27.962500,10,15.139748,12.001957,17.624052,1.468432,5.333333
140516,4000.000000,None,None,"LINESTRING (150015.142 -449929.231, 150013.134...",8570002_JUNE23,10857000020825-JUNE23,1.0,857-13168,857,0.0,...,1000.000000,5.681818,NaN,12.710227,10,15.139748,12.001957,17.624052,1.468432,5.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117432,53259.994662,7148,Victory / Owensmouth,"LINESTRING (128549.034 -424467.506, 128556.908...",1690143_JUNE23,10169001430730-JUNE23,132.0,169-13168,169,1.0,...,376.657210,10.761635,-85.0,24.073777,3,15.046110,12.759102,20.462710,1.603774,1.666667
117433,53259.994662,7148,Victory / Owensmouth,"LINESTRING (128549.034 -424467.506, 128556.908...",1690143_JUNE23,10169001430533-JUNE23,132.0,169-13168,169,1.0,...,376.657210,5.022096,0.0,11.234429,3,15.046110,12.759102,20.462710,1.603774,1.666667
93776,53777.670105,12111,Hollywood Way / Thornton,"LINESTRING (152149.885 -422713.250, 152149.886...",1690148_JUNE23,10169001480702-JUNE23,133.0,169-13168,169,0.0,...,480.900946,3.847208,66.0,8.606203,3,10.148825,9.223252,16.968835,1.839789,1.333333
93777,53777.670105,12111,Hollywood Way / Thornton,"LINESTRING (152149.885 -422713.250, 152149.886...",1690148_JUNE23,10169001480606-JUNE23,133.0,169-13168,169,0.0,...,480.900946,9.618019,-10.0,21.515508,3,10.148825,9.223252,16.968835,1.839789,1.333333


## Side-by-Side Maps

In [117]:
df_eric.shape, df_tiff.shape 

((151014, 23), (155314, 24))

In [118]:
#df_tiff.route_id.unique()

In [119]:
#one_route = "96-13168"
#df_tiff[df_tiff.route_id==one_route].explore(
#    "route_id", tiles = "CartoDB Positron")

In [120]:
#df_tiff[df_tiff.route_id==one_route].trip_id.unique()

In [121]:
trips_to_try = {
    "metro_720": "10720012750651-JUNE23", #route_id: 720-13168
    "metro_901": "10901000590843-JUNE23", #route_id: 901-13168
    "metro_550": "10550001350610-JUNE23", # route_id: 550-13168
    "metro_230":  "10230000830600-JUNE23", # route_id: 230-13168
    "metro_96": "10096002510743-JUNE23", # route_id: 96-13168
    "bbb1": "908521", # route_id: 3639
    #"bbb2": "", #route_id
}

In [122]:
# map_one_trip(df_eric, trips_to_try["metro_720"])

In [123]:
# map_one_trip(df_tiff, trips_to_try["metro_720"])

In [124]:
# map_one_trip(df_eric, trips_to_try["metro_901"])

In [125]:
# map_one_trip(df_tiff, trips_to_try["metro_901"])

In [126]:
# map_one_trip(df_eric, trips_to_try["metro_550"])

In [127]:
# map_one_trip(df_tiff, trips_to_try["metro_550"])

In [128]:
# map_one_trip(df_eric, trips_to_try["metro_230"])

In [129]:
# map_one_trip(df_tiff, trips_to_try["metro_230"])

In [130]:
map_one_trip(df_eric, trips_to_try["metro_96"])

In [131]:
map_one_trip(df_tiff, trips_to_try["metro_96"])

In [132]:
# map_one_trip(df_eric, trips_to_try["bbb1"])

In [133]:
# map_one_trip(df_tiff, trips_to_try["bbb1"])

# Dig deeper into a specific LA Metro G Line (901) segment

* big speed difference, but...
* not actually the same segment, Tiff is stop to stop but Eric cuts an interpolated segment since distance between stops >1000m
* interesting case where Eric _slower_ than Tiff when general pattern is opposite, but due to segment mismatch maybe not the best to look at first

In [134]:
trip_segment = {'trip_id': trips_to_try['metro_901'], 'stop_sequence': 8}

In [135]:
filter_trip = filter(_.trip_id == trip_segment['trip_id'], _.stop_sequence == trip_segment['stop_sequence'])

In [136]:
df_eric >> filter_trip

,shape_meters,stop_id,stop_name,geometry,shape_id,trip_id,stop_sequence,route_id,route_short_name,direction_id,...,meters_from_last,speed_from_last,delay_chg_sec,speed_mph,n_trips_shp,p50_mph,p20_mph,p80_mph,fast_slow_ratio,trips_per_hour
109199,11377.078239,15606,Woodley Station,"LINESTRING (139447.609 -424394.891, 139508.002...",9010059_JUNE23,10901000590843-JUNE23,8.0,901-13168,None,0.0,...,377.078239,2.582728,NaN,5.777562,9,12.224986,6.458984,16.165203,2.502747,9.0


In [137]:
(df_eric >> filter_trip).explore()

In [138]:
df_tiff >> filter_trip

,schedule_gtfs_dataset_key,shape_array_key,geometry,stop_id,stop_sequence,loop_or_inlining,seg_idx,district,trip_instance_key,segment_meters,...,meters_elapsed,sec_elapsed,pct_segment,speed_mph,flag_recalculated,name,trip_id,shape_id,route_id,direction_id
17626,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,20c826dfb5a463334e8942cdb3e795d5,"LINESTRING (138216.368 -424448.224, 138216.382...",15606,8,0,25889,7,1ee881f8148d699747dbcd0859fcd7f6,1612.503377,...,1605.959777,259,0.995942,13.87078,0,LA Metro Bus Schedule,10901000590843-JUNE23,9010059_JUNE23,901-13168,0.0


In [139]:
(df_tiff >> filter_trip).explore()

# Dig deeper into a specific LA Metro 96 segment

* good simple example, same exact segment geom!
* Tiff: 2.93mph, Eric: 7.95mph (over 2x!)
* so... why?
* partial answer -- Tiff assigns dwell time at entry stop entirely to this seg, Eric interpolates and splits dwell between this seg and previous.

In [140]:
trip_segment = {'trip_id': trips_to_try['metro_96'], 'stop_sequence': 36}

In [141]:
filter_trip = filter(_.trip_id == trip_segment['trip_id'], _.stop_sequence == trip_segment['stop_sequence'])

In [142]:
df_eric >> filter_trip

,shape_meters,stop_id,stop_name,geometry,shape_id,trip_id,stop_sequence,route_id,route_short_name,direction_id,...,meters_from_last,speed_from_last,delay_chg_sec,speed_mph,n_trips_shp,p50_mph,p20_mph,p80_mph,fast_slow_ratio,trips_per_hour
36414,16647.487169,18094,Riverside / Riverdale,"LINESTRING (162553.508 -434619.519, 162553.710...",960251_JUNE23,10096002510743-JUNE23,36.0,96-13168,96,1.0,...,184.912802,3.556015,-8.0,7.954807,4,10.440684,7.836958,15.634983,1.995032,1.333333


In [143]:
(df_eric >> filter_trip).explore()

In [144]:
df_tiff >> filter_trip

,schedule_gtfs_dataset_key,shape_array_key,geometry,stop_id,stop_sequence,loop_or_inlining,seg_idx,district,trip_instance_key,segment_meters,...,meters_elapsed,sec_elapsed,pct_segment,speed_mph,flag_recalculated,name,trip_id,shape_id,route_id,direction_id
70379,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,7a4a0395f4918ece6eb0c18164b8c053,"LINESTRING (162553.508 -434619.519, 162553.710...",18094,36,0,93587,7,edfe9a6af705fe4181d3b916df2fdddf,184.912802,...,181.149217,138,0.979647,2.936455,0,LA Metro Bus Schedule,10096002510743-JUNE23,960251_JUNE23,96-13168,1.0


In [145]:
(df_tiff >> filter_trip).explore()

## How does `rt_parser` calculate _this segment_??

In [146]:
from rt_analysis import rt_parser
from tqdm.notebook import tqdm
import datetime as dt

In [147]:
pbar = tqdm()

0it [00:00, ?it/s]

In [148]:
%%capture
analysis_date_dt = dt.date.fromisoformat(analysis_date)
rt_day = rt_parser.OperatorDayAnalysis(182, analysis_date_dt, pbar,
                                                   debug_trip_list=[trips_to_try['metro_96']])

In [149]:
def get_stop_segment(rt_parser, ending_stop_id):
    trs = rt_parser.trs.copy()
    this_stop = trs >> filter(_.stop_id == ending_stop_id)
    prev_stop = trs >> filter(_.shape_meters < this_stop.shape_meters.iloc[0]) >> filter(_.shape_meters == _.shape_meters.max())
    return pd.concat([prev_stop, this_stop])

In [150]:
def select_interpolation_points(df, shape_meters):
    point1 = df >> filter(_.shape_meters <= shape_meters) >> filter(_.shape_meters == _.shape_meters.max())
    point2 = df >> filter(_.shape_meters > shape_meters) >> filter(_.shape_meters == _.shape_meters.min())
    return pd.concat([point1, point2])

In [151]:
def select_interpolation_sets(df, segment_df):
    first_set = select_interpolation_points(df, segment_df.shape_meters.min())
    second_set = select_interpolation_points(df, segment_df.shape_meters.max())
    return pd.concat([first_set, second_set])

In [152]:
def select_speedmap_points(trip_segment_dict, rt_day):
    '''
    return 4 points used to interpolate segment.
    TODO: assert this and prior stop sequence are integers
    '''
    st_entry = rt_day.stop_times >> filter(_.trip_id == trip_segment_dict["trip_id"], _.stop_sequence == trip_segment_dict["stop_sequence"])
    stop_id = st_entry.stop_id.iloc[0]
    speedmap_segment = get_stop_segment(rt_day, stop_id)
    
    interpolator = rt_day.position_interpolators[trip_segment_dict["trip_id"]]['rt']
    speedmap_segment['str_time'] = speedmap_segment.shape_meters.map(lambda x: interpolator.time_at_position(x))
    speedmap_segment['str_time'] = speedmap_segment.str_time.apply(lambda x: x.isoformat())
    speedmap_segment['observation_type'] = 'speedmap_stop_interpolated'
    # display(speedmap_segment)
    df = interpolator.cleaned_positions.copy()
    df['str_time'] = df.location_timestamp_local.apply(lambda x: x.isoformat())
    df =(df >> select(-_.location_timestamp_local)
            >> select(_.gtfs_dataset_name, _.trip_instance_key, _.shape_meters,
                      _.meters_from_last, _.secs_from_last, _.speed_from_last,
                     _.str_time, _.geometry)
        )
    df['observation_type'] = 'speedmap_interpolation_source'
    interpolation_sets = select_interpolation_sets(df, speedmap_segment)
    
    df = pd.concat([interpolation_sets, speedmap_segment])
    for col in ['gtfs_dataset_name', 'trip_instance_key', 'shape_id']:
        df[col].ffill(inplace=True)
        df[col].bfill(inplace=True)
    
    return df

## How does `rt_segment_speeds` calculate _this segment_??

In [153]:
def select_segment_points(trip_segment_dict):

    trip = helpers.import_scheduled_trips(
    analysis_date,
    columns = ["trip_instance_key", "trip_id"],
    filters = [[("trip_id", "==", trip_segment_dict['trip_id'])]],
    get_pandas = True
    )

    trip_key = trip.trip_instance_key.iloc[0]
    vp_pared = pd.read_parquet(
        f"{SEGMENT_GCS}vp_pared_stops_{analysis_date}",
        filters = [[("trip_instance_key", "==", trip_key)]])
    
    # 27:13, 27:56  *  29:14, 29:52, * 30:13
    seg_df = vp_pared[vp_pared.stop_sequence==trip_segment_dict['stop_sequence']]
    seg_gdf = gpd.GeoDataFrame(seg_df, geometry=gpd.points_from_xy(seg_df.x, seg_df.y),
                              crs='EPSG:4326').to_crs(PROJECT_CRS)

    seg_gdf['observation_type'] = 'segment_spd_position'
    seg_gdf['str_time'] = seg_gdf.location_timestamp_local.apply(lambda x: x.isoformat())
    seg_gdf = seg_gdf >> select(-_.location_timestamp_local)
    
    return seg_gdf

In [154]:
seg_gdf = select_segment_points(trip_segment)
seg_gdf

,vp_idx,trip_instance_key,x,y,shape_array_key,stop_sequence,geometry,observation_type,str_time
__null_dask_index__,,,,,,,,,
987109,3716846,edfe9a6af705fe4181d3b916df2fdddf,-118.238464,34.090410,7a4a0395f4918ece6eb0c18164b8c053,36,POINT (162543.436 -434627.916),segment_spd_position,2023-09-13T08:27:34
3173860,3716853,edfe9a6af705fe4181d3b916df2fdddf,-118.237050,34.089252,7a4a0395f4918ece6eb0c18164b8c053,36,POINT (162676.285 -434753.929),segment_spd_position,2023-09-13T08:29:52


## Combined spot-check + map + calculate tool for single segments

In [155]:
trip_segment

{'trip_id': '10096002510743-JUNE23', 'stop_sequence': 36}

In [206]:
trip_segment['stop_sequence'] = 36

In [207]:
def display_segment_speeds(trip_segment_dict, df_speedmap=df_eric, df_segments=df_tiff):
    filter_trip = filter(_.trip_id == trip_segment_dict['trip_id'], _.stop_sequence == trip_segment_dict['stop_sequence'])
    select_cols = select(_.stop_name, _.stop_id, _.shape_id, _.speed_mph)
    df_speedmap = df_speedmap >> filter_trip >> select(_.stop_name, _.stop_id, _.shape_id, _.speed_mph, _.stop_sequence)
    df_segments = df_segments >> filter_trip >> select(_.stop_id, _.shape_id, _.speed_mph, _.stop_sequence)
    print('speedmap:')
    display(df_speedmap)
    print('segments:')
    display(df_segments)

In [208]:
rt_day.shapes

,feed_key,shape_id,geometry,km_index
0,edb0fd72b98a7f739407ceca3fae9034,400949_JUNE23,"LINESTRING (152013.188 -459465.812, 152041.996...",[]
1,9efa698d2ab5a35cb38d49a77e6ea1bc,804WB_RC_221121,"LINESTRING (170545.635 -440800.541, 170457.977...",[]
2,edb0fd72b98a7f739407ceca3fae9034,7540100_JUNE23,"LINESTRING (157606.984 -433671.080, 157603.985...",[]
3,edb0fd72b98a7f739407ceca3fae9034,940256_JUNE23,"LINESTRING (149770.224 -426173.721, 149840.540...",[]
4,edb0fd72b98a7f739407ceca3fae9034,2340074_JUNE23,"LINESTRING (141418.487 -427906.615, 141419.534...",[]
...,...,...,...,...
625,edb0fd72b98a7f739407ceca3fae9034,9010054_JUNE23,"LINESTRING (149522.147 -426193.810, 149572.123...",[]
626,edb0fd72b98a7f739407ceca3fae9034,7201299_JUNE23,"LINESTRING (139445.861 -443521.791, 139430.952...",[]
627,9efa698d2ab5a35cb38d49a77e6ea1bc,807SB_220810,"LINESTRING (153779.228 -442350.572, 153779.456...",[]
628,edb0fd72b98a7f739407ceca3fae9034,9010055_JUNE23,"LINESTRING (128988.272 -417250.364, 128988.310...",[]


In [209]:
def combine_and_map(trip_segment_dict, rt_day=rt_day):
    display_segment_speeds(trip_segment_dict)
    speedmap_gdf = select_speedmap_points(trip_segment_dict, rt_day)
    segment_gdf = select_segment_points(trip_segment_dict)
    this_shape = rt_day.shapes >> filter(_.shape_id == speedmap_gdf.shape_id.iloc[0])
    segment_gdf['shape_meters'] = segment_gdf.geometry.map(lambda x: this_shape.geometry.iloc[0].project(x))
    m_speedmap = speedmap_gdf.explore(column='observation_type', cmap=CALITP_CATEGORY_BOLD_COLORS, name='speedmap')
    m_segment = segment_gdf.explore(m=m_speedmap, column='observation_type', cmap=CALITP_CATEGORY_BOLD_COLORS[3:],
                                   name='segment_speeds')
    folium.LayerControl().add_to(m_segment)
    combined_gdf = pd.concat([speedmap_gdf, segment_gdf])
    # display(combined_gdf)
    # return combined_gdf, combined_gdf.explore(column='observation_type', cmap=CALITP_CATEGORY_BOLD_COLORS)
    return combined_gdf, m_segment

In [210]:
gdf36, m = combine_and_map(trip_segment)

speedmap:


,stop_name,stop_id,shape_id,speed_mph,stop_sequence
36414,Riverside / Riverdale,18094,960251_JUNE23,7.954807,36.0


segments:


,stop_id,shape_id,speed_mph,stop_sequence
70379,18094,960251_JUNE23,2.936455,36


/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


In [211]:
m

In [201]:
gdf36

,gtfs_dataset_name,trip_instance_key,shape_meters,meters_from_last,secs_from_last,speed_from_last,str_time,geometry,observation_type,stop_id,stop_name,shape_id,vp_idx,x,y,shape_array_key,stop_sequence
99,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16461.683094,118.415611,43.0,2.753851,2023-09-13T08:27:56,POINT (162543.436 -434627.916),speedmap_interpolation_source,NaN,NaN,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
194,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16462.749086,1.065992,78.0,0.013667,2023-09-13T08:29:14,POINT (162543.816 -434628.464),speedmap_interpolation_source,NaN,NaN,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
150,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16643.723584,42.407242,18.0,2.355958,2023-09-13T08:29:52,POINT (162676.285 -434753.929),speedmap_interpolation_source,NaN,NaN,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
184,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16688.414044,44.690460,21.0,2.128117,2023-09-13T08:30:13,POINT (162711.880 -434782.004),speedmap_interpolation_source,NaN,NaN,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
6,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16462.574366,NaN,NaN,NaN,2023-09-13T08:29:01.215570,POINT (162545.816 -434626.873),speedmap_stop_interpolated,18093,Riverside / Dorris,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
25,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16647.487169,NaN,NaN,NaN,2023-09-13T08:29:53.768505,POINT (162680.185 -434755.188),speedmap_stop_interpolated,18094,Riverside / Riverdale,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
987109,NaN,edfe9a6af705fe4181d3b916df2fdddf,16461.683094,NaN,NaN,NaN,2023-09-13T08:27:34,POINT (162543.436 -434627.916),segment_spd_position,NaN,NaN,NaN,3716846.0,-118.238464,34.090410,7a4a0395f4918ece6eb0c18164b8c053,36.0
3173860,NaN,edfe9a6af705fe4181d3b916df2fdddf,16643.723584,NaN,NaN,NaN,2023-09-13T08:29:52,POINT (162676.285 -434753.929),segment_spd_position,NaN,NaN,NaN,3716853.0,-118.237050,34.089252,7a4a0395f4918ece6eb0c18164b8c053,36.0


In [203]:
gdf35, m = combine_and_map(trip_segment)

speedmap:


,stop_name,stop_id,shape_id,speed_mph,stop_sequence
36410,Riverside / Dorris,18093,960251_JUNE23,3.819054,35.0


segments:


,stop_id,shape_id,speed_mph,stop_sequence
70375,18093,960251_JUNE23,3.372485,35


/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


In [205]:
pd.concat([gdf35, gdf36])

,gtfs_dataset_name,trip_instance_key,shape_meters,meters_from_last,secs_from_last,speed_from_last,str_time,geometry,observation_type,stop_id,stop_name,shape_id,vp_idx,x,y,shape_array_key,stop_sequence
75,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16237.746100,189.516441,25.0,7.580658,2023-09-13T08:26:56,POINT (162404.836 -434450.414),speedmap_interpolation_source,NaN,NaN,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
71,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16343.267483,105.521383,17.0,6.207140,2023-09-13T08:27:13,POINT (162467.331 -434535.575),speedmap_interpolation_source,NaN,NaN,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
99,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16461.683094,118.415611,43.0,2.753851,2023-09-13T08:27:56,POINT (162543.436 -434627.916),speedmap_interpolation_source,NaN,NaN,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
194,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16462.749086,1.065992,78.0,0.013667,2023-09-13T08:29:14,POINT (162543.816 -434628.464),speedmap_interpolation_source,NaN,NaN,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
13,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16254.293354,NaN,NaN,NaN,2023-09-13T08:26:58.665842,POINT (162414.135 -434464.111),speedmap_stop_interpolated,18092,Riverside / Birkdale,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
6,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16462.574366,NaN,NaN,NaN,2023-09-13T08:29:01.215570,POINT (162545.816 -434626.873),speedmap_stop_interpolated,18093,Riverside / Dorris,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
1462461,NaN,edfe9a6af705fe4181d3b916df2fdddf,16237.746100,NaN,NaN,NaN,2023-09-13T08:26:56,POINT (162404.836 -434450.414),segment_spd_position,NaN,NaN,NaN,3716844.0,-118.239930,34.092033,7a4a0395f4918ece6eb0c18164b8c053,35.0
3966148,NaN,edfe9a6af705fe4181d3b916df2fdddf,16462.341135,NaN,NaN,NaN,2023-09-13T08:29:14,POINT (162543.816 -434628.464),segment_spd_position,NaN,NaN,NaN,3716851.0,-118.238460,34.090405,7a4a0395f4918ece6eb0c18164b8c053,35.0
99,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16461.683094,118.415611,43.0,2.753851,2023-09-13T08:27:56,POINT (162543.436 -434627.916),speedmap_interpolation_source,NaN,NaN,960251_JUNE23,NaN,NaN,NaN,NaN,NaN
194,LA Metro Bus Vehicle Positions,edfe9a6af705fe4181d3b916df2fdddf,16462.749086,1.065992,78.0,0.013667,2023-09-13T08:29:14,POINT (162543.816 -434628.464),speedmap_interpolation_source,NaN,NaN,960251_JUNE23,NaN,NaN,NaN,NaN,NaN


In [212]:
speed_df

,trip_id,shape_id,stop_id,stop_sequence,route_id,direction_id,eric_speed_mph,tiff_speed_mph,_merge
0,58736804,801NB_RC_221121,None,1.25,801,0.0,43.453911,NaN,left_only
1,58736805,801NB_RC_221121,None,1.25,801,0.0,64.675588,NaN,left_only
2,58736808,801NB_RC_221121,None,1.25,801,0.0,64.675588,NaN,left_only
3,58736804,801NB_RC_221121,None,1.50,801,0.0,52.023256,NaN,left_only
4,58736805,801NB_RC_221121,None,1.50,801,0.0,57.358974,NaN,left_only
...,...,...,...,...,...,...,...,...,...
151009,908769,26483,1099,14.00,3641,0.0,16.690712,NaN,left_only
151010,908765,26483,1099,14.00,3641,0.0,14.218014,10.602149,both
151011,908767,26483,1099,14.00,3641,0.0,23.265841,NaN,left_only
151012,908768,26483,1099,14.00,3641,0.0,19.686481,7.420008,both
